# RAG


# Overview

검색 증강 생성(Retrieval Augmented Generation)은 대규모 언어 모델이 지식을 생성할 때 외부 지식원을 활용하는 기술을 말합니다.

쉽게 설명하자면, 언어 모델이 문장을 생성할 때 관련된 정보를 인터넷이나 데이터베이스에서 찾아 활용하는 것입니다. 이렇게 하면 모델이 가진 지식의 한계를 넘어 더 정확하고 상세한 내용을 생성할 수 있습니다.

예를 들어 "파리의 인구는 얼마인가?"라는 질문에 대해, 모델은 위키피디아 등에서 파리 인구 통계를 찾아 그 정보를 활용하여 대답할 수 있습니다. 단순히 학습된 지식만으로는 정확한 최신 정보를 제공하기 어렵지만, 외부 지식원을 참고하면 더 나은 결과를 낼 수 있습니다.

이 과정에서는 앞서 생성한 `movie_semantic` 인덱스를 Knowledge Base로 Amazon Bedrock의 Claude V3 Sonnet 모델을 사용하여 환각을 제거하고 정확한 답변을 생성해봅니다.


# 사전준비

이번 단계를 진행하기 위해서는 [시맨틱 검색 단계](./02.semantic_search.ipynb)를 필수적으로 완료하셔야 합니다. Amazon OpenSearch Service로의 연결은 [시맨틱 검색 단계](./02.semantic_search.ipynb)와 동일하게 수행합니다.


패키지를 설치합니다.


In [ ]:
!pip install -q boto3
!pip install -q requests
!pip install -q requests-aws4auth
!pip install -q opensearch-py
!pip install -q tqdm
!pip install -q boto3
!pip install -q langchain
!pip install -q langchain_community
!pip install -U -q langchain-aws

## 환각 발생시키지

RAG를 구성하지 않고 바로 Amazon Bedrock Claude Sonnet V3에게 다음과 같은 질문을 던져 봅니다.


In [1]:
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth
from langchain.vectorstores import OpenSearchVectorSearch
import boto3
import json
import textwrap

In [2]:
query_text = "건축학개론 내용과 평점은?"

In [3]:
from langchain_aws import ChatBedrock
from langchain_core.messages import HumanMessage

# region_name = "us-west-2"
session = boto3.Session()
region_name = session.region_name

model_kwargs = {  # anthropic
    "anthropic_version": "bedrock-2023-05-31",
    "max_tokens": 2048,
    "temperature": 0,
}

llm = ChatBedrock(
    model_id="anthropic.claude-3-sonnet-20240229-v1:0",  # 파운데이션 모델 지정
    model_kwargs=model_kwargs,
    region_name=region_name,
)  # Claude 속성 구성

messages = [HumanMessage(content=query_text)]

print(textwrap.fill(llm.invoke(messages).content, 80))

건축학개론은 건축학을 전공하는 학생들에게 건축의 기본 개념과 원리를 소개하는 과목입니다. 일반적으로 다음과 같은 내용을 다룹니다:  1. 건축의
정의와 역사 2. 건축설계 프로세스  3. 건축 형태와 공간 구성 4. 건축 재료와 구조 5. 환경과 지속가능한 건축 6. 건축 법규와 규정
7. 건축가의 역할과 직업윤리   수강생들은 이 과목을 통해 건축학의 기초지식을 습득하고 건축가로서의 사고방식을 기를 수 있습니다.   평점은
대학과 강의 수준에 따라 다르지만, 대체로 중간 정도의 난이도를 가진 과목으로 평가됩니다. 수강생들의 성실성과 관심도에 따라 좋은 평점을 받을
수 있습니다. 건축 전공자라면 필수적으로 수강해야 하는 기초 과목 중 하나입니다.


Amazon Bedrock Claude Sonnet V3은 한국 영화 건축학 개론에 대한 답을 모르기 때문에 엉뚱한 답변을 하게 됩니다.


## RAG을 통해 환각 없애기


이제 OpenSearch에 저장된 정보를 컨텍스트로 전달하여 환각을 없애보겠습니다. 여기서는 이전 단계에서 사용한 [하이브리드 검색 기법](./03.hybrid_search.ipynb)을 활용해보도록 하겠습니다


## OpenSearch 클라이언트 연결


이전 과정에서 수행했던 것과 동일한 방법으로 OpenSearch 도메인에 연결합니다.


In [4]:
def get_cfn_outputs(stackname, cfn):
    outputs = {}
    for output in cfn.describe_stacks(StackName=stackname)["Stacks"][0]["Outputs"]:
        outputs[output["OutputKey"]] = output["OutputValue"]
    return outputs

먼저 CloudFormation 스택으로보터 호스트와 인증 정보를 가져옵니다.


In [5]:
import boto3, json

region_name = "us-west-2"

cfn = boto3.client("cloudformation", region_name)
kms = boto3.client("secretsmanager", region_name)

stackname = "opensearch-workshop"
cfn_outputs = get_cfn_outputs(stackname, cfn)

aos_credentials = json.loads(
    kms.get_secret_value(SecretId=cfn_outputs["OpenSearchSecret"])["SecretString"]
)

aos_host = cfn_outputs["OpenSearchDomainEndpoint"]

가져온 정보를 바탕으로 OpenSearch 도메인에 연결합니다.


In [6]:
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth

auth = (aos_credentials["username"], aos_credentials["password"])

aos_client = OpenSearch(
    hosts=[{"host": aos_host, "port": 443}],
    http_auth=auth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
)

인덱스명과 모델 아이디를 설정합니다.


In [7]:
import requests

search_model = {"query": {"match": {"name": "OpenSearch-Cohere"}}, "size": 10}

response = requests.get(
    "https://" + aos_host + "/_plugins/_ml/models/_search", auth=auth, json=search_model
)
model_info = json.loads(response.text)
model_id = model_info["hits"]["hits"][0]["_id"]

index_name = "movie_semantic"

## 하이브리드 검색 함수 구현

이전 [하이브리드 검색 단계](https://www.notion.so/Hybrid-Search-9b923c1a7b2e4d1697768c3385ea47d0?pvs=21)에서 사용한 `hybrid_search` 함수를 다음과 같이 변경합니다. 가장 큰 변경 부분은 검색된 결과를 LangChain의 Document 객체로 변환해서 반환한다는 점입니다. 이렇게 생성된 **`Document`** 객체들을 기반으로 LangChain은 다양한 작업을 수행할 수 있습니다.


In [8]:
from langchain.schema import Document
import pandas as pd


def hybrid_search(
    query_text,
    keyword_weight=0.3,
    semantic_weight=0.7,
):
    print(query_text)
    query = {
        "size": 10,
        "_source": {"exclude": ["text", "vector_field"]},
        "query": {
            "hybrid": {
                "queries": [
                    {
                        "multi_match": {
                            "query": query_text,
                            "fields": ["title", "plot", "genre", "main_act", "supp_act"],
                        }
                    },
                    {
                        "neural": {
                            "vector_field": {
                                "query_text": query_text,
                                "model_id": model_id,
                                "k": 30,
                            }
                        }
                    },
                ]
            }
        },
        "search_pipeline": {
            "description": "Post processor for hybrid search",
            "phase_results_processors": [
                {
                    "normalization-processor": {
                        "normalization": {"technique": "min_max"},
                        "combination": {
                            "technique": "arithmetic_mean",
                            "parameters": {"weights": [keyword_weight, semantic_weight]},
                        },
                    }
                }
            ],
        },
    }

    res = aos_client.search(index=index_name, body=query)

    query_result = []
    docs = []
    for hit in res["hits"]["hits"]:
        row = [
            hit["_score"],
            hit["_source"]["title"],
            hit["_source"]["plot"],
            hit["_source"]["genre"],
            hit["_source"]["rating"],
            hit["_source"]["main_act"],
        ]
        query_result.append(row)

        # LangChain에 Context로 제공하기 위한 Document 객체를 준비합니다.
        metadata = {"score": hit["_score"], "id": hit["_id"]}

        content = {
            "제목": hit["_source"]["title"],
            "장르": hit["_source"]["genre"],
            "평점": hit["_source"]["rating"],
            "줄거리": hit["_source"]["plot"],
            "주연": hit["_source"]["main_act"],
            "조연": hit["_source"]["supp_act"],
        }

        doc = Document(page_content=json.dumps(content, ensure_ascii=False), metadata=metadata)

        docs.append(doc)

    query_result_df = pd.DataFrame(
        data=query_result, columns=["_score", "title", "plot", "genre", "rating", "main_act"]
    )
    display(query_result_df)

    return docs

검색 함수가 잘 동작하는지 확인합니다.


In [11]:
docs = hybrid_search(query_text)

print("검색된 문서 총 " + str(len(docs)) + "개")

건축학개론 내용과 평점은?


,_score,title,plot,genre,rating,main_act
0,1.000000,건축학개론,"생기 넘치지만 숫기 없던 스무 살, 건축학과 승민은 '건축학개론' 수업에서 처음 만...",멜로/로맨스,8.67,엄태웅|한가인|이제훈|수지
1,0.226566,고사 두 번째 이야기: 교생실습,"명문 사립 우성 고등학교는 여름 방학이 시작되면, 전교 1등부터 3 등까지 생활관 ...",공포|스릴러,4.73,김수로|황정음|윤시윤|지연|박은빈
2,0.195420,부러진 화살,대학 입시시험에 출제된 수학문제 오류를 지적한 뒤 부당하게 해고된 김경호 교수. 교...,드라마,8.92,안성기|박원상|나영희|김지호
3,0.193144,두 개의 문,"2 9년 1월 2 일, 철거민 5명, 경찰 특공대원 1명 사망. 생존권을 호소하며 ...",다큐멘터리,5.17,권영국|김형태|류주형|박진|박성훈
4,0.190887,작업의 정석,작업계의 대표선수 민준과 지원이 만났다. 선수는 선수를 알아보는 법! 작업계의 고수...,코미디|멜로/로맨스,6.53,손예진|송일국
5,0.189840,천안함 프로젝트,"2 1 년 3월 26일, 대한민국 해군 초계함 ‘PCC-772천안’ 백령도 해상에서...",다큐멘터리,4.34,강신일
6,0.181193,고사: 피의 중간고사,"수능을 약 2 여일 앞둔 어느 토요일, 전교 1등부터 2 등까지의 모범생들만을 위한...",공포|스릴러,6.82,이범수|윤정희|남규리|김범
7,0.164614,베스트셀러,1 여 년간 대한민국 최고의 베스트셀러 작가로 군림해온 백희수. 발표한 신작 소설이...,미스터리,7.60,엄정화
8,0.164321,인사동 스캔들,4 년전 사라졌던 한 그림의 복원 프로젝트가 전국민의 관심 속에 세상에 공개된다. ...,범죄|액션|드라마,7.99,김래원|엄정화
9,0.163321,컨저링,"1971년 로드 아일랜드, 해리스빌. 페론 가족은 꿈에 그리던 새 집으로 이사를 간...",공포|스릴러,7.85,베라 파미가|패트릭 윌슨|릴리 테일러|론 리빙스턴|조이 킹|매켄지 포이


검색된 문서 총 10개


## RAG을 통한 정확한 답변 받기


### 컨텍스트 기반 질문 응답을 위한 프롬프트 템플릿 생성

LangChain의 **`PromptTemplate`**을 사용하여 컨텍스트 기반 질문 응답을 위한 프롬프트 템플릿을 정의합니다.

1. 컨텍스트와 질문을 입력 변수로 받습니다.
2. 컨텍스트는 **`<context>`** 및 **`</context>`** XML 태그로 묶여 제공됩니다.
3. 답변 시 XML 태그를 포함하지 않도록 지시합니다.
4. 답변 시 가능한 한 많은 내용을 포함하도록 지시합니다.
5. 답변 시 공손하고 예의바른 태도를 유지하도록 지시합니다.
6. 컨텍스트에서 답변을 확실히 찾을 수 없는 경우, "주어진 내용에서 관련 답변을 찾을 수 없습니다."라고 답변하도록 지시합니다.


In [12]:
from langchain import PromptTemplate

prompt_template = """


Human: Here is the context, inside <context></context> XML tags.

<context>
{context}
</context>

Only using the contex as above, answer the following question with the rules as below:
    - Don't insert XML tag such as <context> and </context> when answering.
    - Write as much as you can
    - Be courteous and polite
    - Only answer the question if you can find the answer in the context with certainty.

Question:
{question}

If the answer is not in the context, just say "주어진 내용에서 관련 답변을 찾을 수 없습니다."


Assistant:"""

PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

LangChain의 **`load_qa_chain`** 함수를 사용하여 질문 답변 체인을 로드하고, 주어진 문서와 질문에 대한 답변을 생성합니다. 하이브리드 검색으로 찾은 문서 `docs`를 `chain`의 `input_documents`로 제공합니다.


In [13]:
from langchain.chains.question_answering import load_qa_chain

chain = load_qa_chain(llm=llm, chain_type="stuff", prompt=PROMPT)

answer = chain.run(input_documents=docs, question=query_text)

print("query: ", query_text)
answer_str = "answer: \n" + answer
print(textwrap.fill(answer_str, 80))

/opt/anaconda3/envs/opensearch/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


query:  건축학개론 내용과 평점은?
answer:  건축학개론의 줄거리는 "생기 넘치지만 숫기 없던 스무 살, 건축학과 승민은 '건축학개론' 수업에서 처음 만난 음대생 서연에게
반한다. 함께 숙제를 하게 되면서 차츰 마음을 열고 친해지지만, 자신의 마음을 표현하는 데 서툰 순진한 승민은 입 밖에 낼 수 없었던 고백을
마음 속에 품은 채 작은 오해로 인해 서연과 멀어지게 된다. 어쩌면 다시…사랑할 수 있을까? 15년 만에 그녀를 다시 만났다 서른 다섯의
건축가가 된 승민 앞에 15년 만에 불쑥 나타난 서연. 당황스러움을 감추지 못하는 승민에게 서연은 자신을 위한 집을 설계해달라고 한다. 자신의
이름을 건 첫 작품으로 서연의 집을 짓게 된 승민, 함께 집을 완성해 가는 동안 어쩌면 사랑이었을지 모를 그때의 기억이 되살아나 두 사람 사이에
새로운 감정이 쌓이기 시작하는데…"입니다. 평점은 8.67입니다.


위와 같이 정확한 답변이 제공되는 것을 확인할 수 있습니다.
